## 6. 문서 기반 챗봇 (RAG, Retrieval-Augmented Generation)

### 왜 RAG가 중요한가?

기존의 ChatGPT와 같은 **기본 LLM**은 이미 방대한 데이터를 학습했기 때문에  
일반 상식이나 범용 지식에 대해서는 뛰어난 답변을 제공합니다.  
하지만 **두 가지 한계**가 존재합니다.

**1. 최신 정보 반영 불가**  
  - 모델이 학습된 시점 이후의 정보(예: 2024년 기술 논문, 최근 공정 변경, 신소재 데이터)는 알 수 없음
- 예: “2025년 포스코의 신형 TMCP 공정 특성은?” → ChatGPT는 모름

**2. 도메인 특화 지식 부족**
- 용접, 재료, 반도체, 제약, 법률 등 전문 산업의 내부 문서나 보고서 내용은 LLM 학습 데이터에 포함되어 있지 않음
- 예: “9%Ni 강의 GTAW 루트 패스 예열 조건” 같은 질문은 일반 LLM이 정확히 답하지 못함

### RAG (Retrieval-Augmented Generation)란?

**RAG**는 LLM의 한계를 보완하기 위해 외부 문서 검색(Retrieval)과 생성(Generation)을 결합한 기술입니다.  
일반적인 작동 순서는 다음과 같습니다.

**1. 문서 수집 및 벡터화**
- 사내 문서, 논문, 보고서, 표준서 등을 **Embedding(임베딩)** 하여 **벡터DB(Vector Database)** 에 저장합니다.  
  예: “AWS D1.1 용접절차”, “9Ni 용접 WPS”, “LNG 탱크 보고서” 등  

- 하지만 대부분의 문서는 수십 페이지 이상이기 때문에, **한 번에 임베딩하면 검색 품질이 떨어집니다.**  
  따라서 문서를 **의미 단위로 잘라(Split)** 여러 개의 작은 “청크(chunk)”로 나눈 후 임베딩합니다.  
  예를 들어, 100페이지짜리 WPS 문서를 문단 단위로 잘라 500개 정도의 벡터로 저장합니다.

- 이렇게 나누면 LLM이 질문을 받을 때, 전체 문서가 아닌  
  **관련성이 높은 부분(chunk)** 만 찾아서 활용할 수 있으므로  
  **검색 정확도와 응답 효율이 크게 향상됩니다.**

<img src="image/RAG.png" width="600">

**2. 질의(Query) 입력**
- 사용자가 질문을 하면, 그 질문 또한 Embedding 되어 벡터DB의 문서들과 **유사도(Similarity)** 를 비교합니다.  
- 예를 들어, “9%Ni 강의 루트 패스 예열 온도는?” 이라는 질문이 들어오면  
  그와 가장 관련된 문서 조각(Top-k)을 자동으로 검색합니다.

**3. 문서 결합**
- 검색된 문서 내용을 LLM의 프롬프트에 함께 삽입합니다.  
- 이때 “검색 결과를 바탕으로 답변하라”는 지시를 추가하여  
  모델이 추측하지 않고 근거를 참고하도록 합니다.


**4. 답변 생성**
- LLM은 삽입된 문서 내용을 토대로 답변을 생성합니다.  
  즉, 단순히 일반적인 추론이 아니라 **실제 문서 근거 기반 답변**을 제공합니다.  
- 이를 통해 **지엽적이고 전문적인 질문**에도 신뢰도 높은 응답이 가능합니다.

### RAG의 핵심 효과

| 항목 | 기존 LLM | RAG 적용 후 |
|------|-----------|-------------|
| **지식 범위** | 사전 학습된 데이터에 한정 | 사내 문서·논문·데이터까지 확장 |
| **정확성** | 추측성 답변 가능 (Hallucination) | 근거 기반 답변 |
| **갱신성** | 정적 (모델 재학습 필요) | 동적 (벡터DB만 갱신하면 즉시 반영) |
| **비용** | 새로 Fine-tuning 필요 | 모델은 그대로, 문서만 추가 |

#### 예시 ① — 일반 LLM만 사용하는 경우

```bash
질문: 9%Ni 강 LNG 탱크의 루트 패스 용접 시 예열 온도는?  
AI: 9%Ni 강은 극저온용 강재로 알려져 있으며, 일반적으로 저온 인성을 위해 열처리를 수행합니다.
(→ 실제 예열 조건은 제시하지 않음, 추측성 답변)
```

#### 예시 ② — RAG 기반 챗봇의 경우
```bash
질문: 9%Ni 강 LNG 탱크의 루트 패스 용접 시 예열 온도는?  
AI: 이전에 등록된 'POSCO LNG Tank WPS-2024' 문서에 따르면,
루트 패스 예열 온도는 80~120°C로 설정되어 있습니다.
(이전 패스의 수소 집적 방지를 위해 예열 유지가 필요합니다.)
```

→ 이처럼 RAG는 단순히 말로 잘하는 AI가 아니라, **근거를 가진 실무형 전문가 챗봇**을 만드는 기술입니다.

### 정리
- **RAG는 모델을 다시 학습시키지 않고도, 새로운 지식을 즉시 반영**할 수 있는 구조입니다.  
- LLM이 “지식 생성기”라면, RAG는 그 위에 붙는 **“지식 확장기(knowledge extender)”** 입니다.  
- 특히 엔지니어링, 제조, 연구개발 분야에서는  
  사내 표준서·보고서·규격 문서를 기반으로 RAG 챗봇을 구성하면  
  “실무형 QA 비서”를 구현할 수 있습니다.

| 구분 | 설명 |
|------|------|
| **RAG 정의** | Retrieval-Augmented Generation — 검색 기반 생성형 AI |
| **핵심 아이디어** | “모델이 모르면 문서에서 찾아서 답하라” |
| **필요 이유** | LLM의 한계(최신성·전문화 부족) 보완 |
| **핵심 구성요소** | Embedding → Vector DB → Retriever → LLM |
| **적용 예시** | 사내 기술 보고서 챗봇, 논문 기반 QA, 품질 기준 상담, 법률/특허 문서 질의응답 |

#### 삼성 메모리 카드 매뉴얼 PDF 기반 질의응답 예제

In [ ]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain_core.prompts import ChatPromptTemplate
from dotenv import load_dotenv # .env 파일의 환경변수를 자동으로 불러오기 위한 모듈
load_dotenv()  # 실행 시 .env 파일을 찾아 변수들을 환경에 로드

# -----------------------------------------------------
# 1️⃣ PDF 로드
# -----------------------------------------------------
# Samsung_Card_Manual_Korean_1.3.pdf 파일을 읽어옵니다.
# PyPDFLoader는 PDF의 각 페이지를 text로 변환합니다.
loader = PyPDFLoader("data/Samsung_Card_Manual_Korean_1.3.pdf")
pages = loader.load()  # List[Document] 형태로 반환

# -----------------------------------------------------
# 2️⃣ 텍스트 분할 (chunk 단위)
# -----------------------------------------------------
# RecursiveCharacterTextSplitter로 문서를 청크 단위로 쪼갭니다.
# chunk_size와 overlap은 retrieval 성능에 영향을 줍니다.
splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
docs = splitter.split_documents(pages)

# -----------------------------------------------------
# 3️⃣ 임베딩 생성 및 벡터DB 저장
# -----------------------------------------------------
# OpenAI Embeddings를 이용해 각 문서 청크를 벡터화하고
# FAISS에 저장합니다. (로컬 DB로 빠른 검색 가능)
embeddings = OpenAIEmbeddings()
vectordb = FAISS.from_documents(docs, embeddings)

# 검색기(retriever) 객체 생성
retriever = vectordb.as_retriever(search_kwargs={"k": 3})  # 상위 3개 유사 문서 검색

# -----------------------------------------------------
# 4️⃣ 프롬프트 템플릿 정의
# -----------------------------------------------------
# 사용자의 질문과 검색된 문서를 결합해 LLM에 전달할 프롬프트를 구성합니다.
prompt = ChatPromptTemplate.from_template("""
너는 삼성전자 메모리카드 매뉴얼에 대한 전문 어시스턴트이다.
다음의 참고 문서를 바탕으로 질문에 정확하게 답하라.

[참고문서]
{context}

[질문]
{question}

한글로 간결하고 정확하게 답변하라.
""")

In [5]:
# LCEL(Runnable Sequence) 문법을 이용하여 pipeline을 정의
# Retriever로 문서를 검색하고, prompt로 결합한 뒤 LLM으로 생성하는 구조
rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}  # 입력을 question으로 받아 retriever 검색
    | prompt                                                    # prompt에 삽입
    | ChatOpenAI(model="gpt-4o-mini", temperature=0)            # OpenAI LLM 호출
)

# -----------------------------------------------------
# 6️⃣ 질의 수행
# -----------------------------------------------------
# 실제로 사용자가 질문을 던지는 부분입니다.
query = "이 유틸리티는 동시에 몇 개의 메모리카드나 UFD를 인식할 수 있나?"  # 예시 질의
answer = rag_chain.invoke(query)

# -----------------------------------------------------
# 7️⃣ 결과 출력
# -----------------------------------------------------
print("질문:", query)
print("답변:", answer.content)

질문: 이 유틸리티는 동시에 몇 개의 메모리카드나 UFD를 인식할 수 있나?
답변: 이 유틸리티는 동시에 최대 8개의 메모리 카드나 UFD를 인식할 수 있습니다.


#### ChatGPT vs RAG 챗봇 성능 비교 테스트

In [6]:
llm_base = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# ANSI 색상 코드
RED = "\033[91m"
BLUE = "\033[94m"
RESET = "\033[0m"


# 비교할 질문 목록
questions = [
    "삼성 메모리 카드/UFD 인증 유틸리티에서 동시에 몇 개의 카드나 UFD를 인식할 수 있습니까?",
    "삼성 메모리 카드/UFD 인증 유틸리티는 BitLocker가 활성화된 장치나 포맷되지 않은 장치를 인증할 수 있습니까?",
    "삼성 메모리 카드/UFD 인증 유틸리티에서 지원되는 운영체제(OS) 버전은 무엇입니까?"
]
#성능 비교 실행

for i, q in enumerate(questions, 1):
    print(f"\n=== 질문 {i} ===")
    print("Q:", q)
    
    # 기본 ChatGPT (비RAG)
    base_answer = llm_base.invoke(q)
    print(f"\n{RED}[일반 ChatGPT 답변]{RESET}")
    print(base_answer.content)
    
    # RAG 기반 ChatGPT
    rag_answer = rag_chain.invoke(q)
    print(f"\n{BLUE}[RAG 기반 답변]{RESET}")
    print(rag_answer.content)
    
    print("\n----------------------------------")


=== 질문 1 ===
Q: 삼성 메모리 카드/UFD 인증 유틸리티에서 동시에 몇 개의 카드나 UFD를 인식할 수 있습니까?

[일반 ChatGPT 답변]
삼성 메모리 카드 및 UFD(USB 플래시 드라이브) 인증 유틸리티는 일반적으로 여러 개의 메모리 카드나 UFD를 동시에 인식할 수 있습니다. 그러나 동시에 인식할 수 있는 개수는 사용 중인 컴퓨터의 USB 포트 수와 연결된 장치의 종류에 따라 달라질 수 있습니다. 일반적으로 USB 포트가 여러 개 있는 경우, 여러 개의 UFD를 동시에 연결하여 인증을 진행할 수 있습니다.

정확한 동시 인식 개수는 유틸리티의 버전이나 사용 환경에 따라 다를 수 있으므로, 구체적인 정보는 삼성의 공식 문서나 고객 지원을 통해 확인하는 것이 좋습니다.

[RAG 기반 답변]
삼성 메모리 카드/UFD 인증 유틸리티는 동시에 최대 8개의 카드를 인식할 수 있습니다.

----------------------------------

=== 질문 2 ===
Q: 삼성 메모리 카드/UFD 인증 유틸리티는 BitLocker가 활성화된 장치나 포맷되지 않은 장치를 인증할 수 있습니까?

[일반 ChatGPT 답변]
삼성 메모리 카드/UFD 인증 유틸리티는 일반적으로 BitLocker가 활성화된 장치나 포맷되지 않은 장치를 인증할 수 없습니다. BitLocker가 활성화된 장치는 암호화된 상태이기 때문에 인증 유틸리티가 해당 장치의 내용을 읽거나 검증하는 데 어려움이 있을 수 있습니다. 또한 포맷되지 않은 장치는 파일 시스템이 없기 때문에 인증을 위한 필요한 정보를 제공하지 않습니다.

따라서 인증을 원하신다면, BitLocker를 비활성화하거나 장치를 포맷한 후에 인증 유틸리티를 사용해야 할 것입니다.

[RAG 기반 답변]
삼성 메모리 카드/UFD 인증 유틸리티는 BitLocker가 활성화된 장치와 포맷되지 않은 장치를 인증할 수 없습니다.

----------------------------------

=== 질문 3 ===
Q:

##  7. 웹 UI 연동
### 🔹 Streamlit이란?
- 파이썬 코드만으로 손쉽게 웹 애플리케이션을 만들 수 있는 프레임워크  
- 데이터 과학, 머신러닝 프로젝트의 **UI 제작에 최적화**  
- 복잡한 프론트엔드 기술(HTML, JS) 없이도 대화형 웹 페이지 제작 가능  

### 🔹 실행 방법
```bash
streamlit run app.py
```
- `--server.port 8501` : 포트 지정  
- `--server.address 0.0.0.0` : 외부 접속 허용  

→ 예:  
```bash
streamlit run app.py --server.port 8501 --server.address 0.0.0.0
```

- 접속: `http://localhost:8501`  
- 외부 네트워크: `http://<서버IP>:8501`

#### Streamlit 실행 테스트

1. `app.py` 파일을 생성하고 다음 내용을 입력한다.

```python
import streamlit as st

st.title("Hello Streamlit 👋")
st.write("Streamlit 앱이 정상적으로 실행되고 있습니다.")
```

2. **터미널**에서 실행한다.

```bash
streamlit run app.py
```

3. 브라우저가 자동으로 열리며 `localhost:8501`에서 앱이 실행되는지 확인한다.

4. 코드 수정 후 저장하면 브라우저가 자동으로 새로고침되는지 확인한다.

### Streamlit 기본 구조 및 구성요소

#### 입력 위젯 (Input Widgets)

사용자 입력을 받는 UI 요소들이다.  
모든 입력 위젯은 Streamlit 내부적으로 **자동 상태 저장(stateful)** 구조를 가진다.

| 위젯 | 설명 | 예시 코드 |
|------|------|-----------|
| `st.text_input()` | 문자열 입력 | `name = st.text_input("이름 입력")` |
| `st.number_input()` | 숫자 입력 | `age = st.number_input("나이", 0, 120)` |
| `st.slider()` | 범위 선택 | `level = st.slider("난이도", 1, 10, 5)` |
| `st.selectbox()` | 드롭다운 선택 | `lang = st.selectbox("언어", ["Python", "R", "C++"])` |
| `st.checkbox()` | 체크박스 | `agree = st.checkbox("동의합니다")` |
| `st.file_uploader()` | 파일 업로드 | `file = st.file_uploader("CSV 업로드")` |

### 예시

```python
import streamlit as st

st.header("입력 위젯 데모")

name = st.text_input("이름을 입력하세요")
age = st.number_input("나이", min_value=0, max_value=120, value=25)
lang = st.selectbox("언어 선택", ["Python", "R", "C++"])
submit = st.button("확인")

if submit:
    st.success(f"{name}님은 {age}세이며, {lang}을(를) 사용합니다.")
```

### 출력 요소 (Output Components)
 
Streamlit은 데이터를 텍스트, 표, 차트 등 다양한 형태로 표현할 수 있습니다.

| 출력 함수 | 용도 | 예시 |
|------------|------|------|
| `st.write()` | 텍스트, 변수, 데이터프레임 자동 처리 | `st.write(df)` |
| `st.dataframe()` | 스크롤 가능한 표 | `st.dataframe(df)` |
| `st.table()` | 정적 표 | `st.table(df.head())` |
| `st.json()` | JSON 포맷 출력 | `st.json({\"name\": \"AI\", \"ver\": 1.0})` |
| `st.line_chart()` | 선형 차트 | `st.line_chart(df)` |
| `st.bar_chart()` | 막대 차트 | `st.bar_chart(df)` |

```python
import streamlit as st
import pandas as pd
import numpy as np

st.header("출력 요소 예제")

data = pd.DataFrame({
    x: np.arange(1, 6),
    y: np.random.randint(1, 100, 5)
})

st.dataframe(data)
st.bar_chart(data.set_index(x))
```

### 디자인 템플릿 및 UI 활용 팁 (NEW)

이 장에서는 Streamlit 앱의 **디자인 완성도와 사용자 경험(UX)** 을 향상시키는 기법을 학습합니다.

디자인은 처음부터 새로 만들기보다, **기존 템플릿을 참고하고 커스터마이징**하는 것이 훨씬 효율적입니다.

 **원칙:**  
1. 처음에는 완성된 구조를 복제하라.  
2. 필요할 때만 커스터마이징하라.  
3. 시각보다는 기능과 명료성을 우선하라.

### 참고 자료
- Streamlit 공식 갤러리: [https://streamlit.io/gallery](https://streamlit.io/gallery)
- GitHub: `awesome-streamlit`, `streamlit-extras` 등
- 커뮤니티 오픈소스: HuggingFace Spaces, Streamlit Community Cloud

#### 🔹 참고 : 추후 배포 방법
1. **도메인 구매** (가비아, GoDaddy, AWS Route53 등)  
2. **클라우드 서버** (AWS, GCP, Naver Cloud 등)에 배포  
3. **Nginx/Apache Reverse Proxy 설정**  
   - `http://chat.my-domain.com` 으로 접근 가능하게 구성

##  8. 복습문제

### 문제 1. API 키를 `.env` 파일에 보관하는 이유는 무엇인가요? 
<details>
<summary>정답 보기</summary>  

- **보안성**: 키 하드코딩/깃 커밋 방지 → 외부 유출 리스크 감소
- **환경 분리**: 로컬/스테이징/프로덕션 별로 키 교체 용이
- **자동화**: CI/CD에서 시크릿 관리와 연동 쉬움
</details>

In [ ]:
### 여기에 정답을 작성해보세요

### 문제 2. **프롬프트 설계**: 

"너는 용접 전문가야"라는 시스템 프롬프트를 넣고, 사용자가 배관 부식 원인을 물었을 때 기술적인 답변을 출력하는 체인을 작성해보세요.  

<details>
<summary>정답 보기</summary>

```python 
from dotenv import load_dotenv              # .env 파일에서 환경변수(OPENAI_API_KEY) 불러오기
import os                                   # OS 환경변수 접근
from langchain_core.prompts import ChatPromptTemplate  # 프롬프트 템플릿(변수 삽입/재사용)
from langchain_openai import ChatOpenAI                # OpenAI 대화형 LLM 래퍼
from langchain_core.output_parsers import StrOutputParser  # 모델 응답을 문자열로 파싱

# ===========================================
# 2) 환경변수 로드
# ===========================================
load_dotenv()                               # 현재 작업 디렉토리의 .env를 로드
# os.getenv("OPENAI_API_KEY")               # 내부적으로 ChatOpenAI가 환경변수를 읽음

# ===========================================
# 3) 프롬프트 템플릿 정의
#    - System: 모델 역할을 '용접 전문가'로 고정
#    - User: 실습 시 입력될 질문 자리표시자 {question}
# ===========================================
prompt = ChatPromptTemplate.from_messages([
    ("system", "너는 용접 전문가이자 재료공학 엔지니어야. 한국어로 간결하지만 정확하게 답변해. "
               "표준/코드(AWS/ASME/KS)나 공정 용어는 정확히 사용해."),
    ("user", "{question}")
])

# ===========================================
# 4) 모델 & 파서 준비
# ===========================================
model = ChatOpenAI(model="gpt-3.5-turbo")   # 필요 시 gpt-4o 등으로 교체 가능
parser = StrOutputParser()                   # 메시지를 최종 문자열로 변환

# ===========================================
# 5) LCEL로 체인 구성: Prompt -> Model -> Parser
# ===========================================
chain = prompt | model | parser

# ===========================================
# 6) 실행 예시
# ===========================================
question = "SUS304 배관에서 담수 조건 핀홀 부식 원인을 설명해줘. 예방책도 제시해줘."
answer = chain.invoke({"question": question})  # 자리표시자에 question 바인딩
print(answer)                                  # 최종 텍스트 출력
```
</details>

In [7]:
### 여기에 정답을 작성해보세요
from dotenv import load_dotenv              # .env 파일에서 환경변수(OPENAI_API_KEY) 불러오기
import os                                   # OS 환경변수 접근
from langchain_core.prompts import ChatPromptTemplate  # 프롬프트 템플릿(변수 삽입/재사용)
from langchain_openai import ChatOpenAI                # OpenAI 대화형 LLM 래퍼
from langchain_core.output_parsers import StrOutputParser  # 모델 응답을 문자열로 파싱

### 문제 3. **대화 기억**: 

사용자의 첫 질문에 "용접의 종류"를 답변한 뒤, 두 번째 질문에 "방금 말한 종류 중 하나를 설명해"라고 했을 때  
올바르게 이어질 수 있도록 메모리를 적용해보세요. 

<details>
<summary>정답 보기</summary>

```python 
from dotenv import load_dotenv
import os
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder  # 히스토리용 플레이스홀더
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

# ===========================================
# 2) 환경변수 로드
# ===========================================
load_dotenv()

# ===========================================
# 3) 대화 히스토리 컨테이너 (간단 리스트)
#    - (role, content) 튜플 목록으로 관리
#    - 실제 프로덕션에선 ConversationBufferMemory 등을 사용할 수 있음
# ===========================================
history = []  # 예: [("user","..."), ("assistant","..."), ...]

# ===========================================
# 4) 프롬프트 템플릿
#    - MessagesPlaceholder("history")를 이용해 멀티턴 히스토리 삽입
# ===========================================
prompt = ChatPromptTemplate.from_messages([
    ("system", "너는 용접 전문가야. 한국어로 단계적으로 설명해."),
    MessagesPlaceholder(variable_name="history"),  # 여기에 이전 턴들 삽입
    ("user", "{input}")                             # 현재 사용자 질문
])

# ===========================================
# 5) 모델 & 파서 & 체인 구성
# ===========================================
model = ChatOpenAI(model="gpt-3.5-turbo")
parser = StrOutputParser()
chain = prompt | model | parser

# ===========================================
# 6) 1차 질문/응답
# ===========================================
user_q1 = "용접의 대표적인 공정 3가지를 요약해줘."
assistant_a1 = chain.invoke({"history": history, "input": user_q1})
print("Assistant(1):", assistant_a1)

# 히스토리 업데이트 (다음 턴에 맥락으로 사용)
history.append(("user", user_q1))
history.append(("assistant", assistant_a1))

# ===========================================
# 7) 2차 질문/응답 (이전 답을 참조하도록 유도)
# ===========================================
user_q2 = "방금 말한 3가지 중 FCAW를 장단점 중심으로 자세히 설명해줘."
assistant_a2 = chain.invoke({"history": history, "input": user_q2})
print("Assistant(2):", assistant_a2)

# 히스토리 업데이트(선택)
history.append(("user", user_q2))
history.append(("assistant", assistant_a2))

```
</details>

In [8]:
### 여기에 정답을 작성해보세요
from dotenv import load_dotenv
import os
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

### 문제 4. **문서 기반 챗봇 확장**: FAISS에 여러 PDF 문서를 임포트해 검색 가능한 챗봇을 만들어보세요.

<details>
<summary>정답 보기</summary>

```python 
# ===========================================
# 1) 라이브러리 임포트
# ===========================================
from dotenv import load_dotenv
import os
from langchain.text_splitter import CharacterTextSplitter        # 문서 청크 분할
from langchain_community.vectorstores import FAISS               # 벡터DB로 FAISS 사용
from langchain_openai import OpenAIEmbeddings                    # OpenAI 임베딩
from langchain_core.prompts import ChatPromptTemplate            # RAG 프롬프트
from langchain_openai import ChatOpenAI                          # LLM
from langchain_core.output_parsers import StrOutputParser        # 출력 파서

# (선택) PDF 로더를 쓰려면 pypdf 설치 후 아래 임포트 가능
# from langchain_community.document_loaders import PyPDFLoader   # pip install pypdf

# ===========================================
# 2) 환경변수 로드
# ===========================================
load_dotenv()

# ===========================================
# 3) 원문 수집
#    - 간단히 텍스트 리스트로 예시
#    - PDF 사용 시: loader = PyPDFLoader("file.pdf"); docs = loader.load()
# ===========================================
raw_texts = [
    "SUS304는 오스테나이트계 스테인리스로 내식성이 우수하나 염화이온 환경에선 틈부식 위험이 존재한다.",
    "FCAW(Flux-Cored Arc Welding)는 고생산성 공정으로 야외 시공에서 이점이 있지만 슬래그 제거가 필요하다.",
    "ASME Section IX는 용접 절차 및 성능시험에 대한 요건을 규정한다."
]

# ===========================================
# 4) 텍스트 → Document로 변환 및 분할
# ===========================================
splitter = CharacterTextSplitter(chunk_size=300, chunk_overlap=50)
documents = splitter.create_documents(raw_texts)  # 리스트[str] → 리스트[Document]

# ===========================================
# 5) 임베딩 & 벡터DB 색인
# ===========================================
embeddings = OpenAIEmbeddings()           # OpenAI 임베딩 사용
vectordb = FAISS.from_documents(documents, embeddings)  # 인메모리 FAISS 인덱스 생성
retriever = vectordb.as_retriever(search_kwargs={"k": 3})  # 상위 3개 유사 청크 검색

# ===========================================
# 6) 간단한 RAG 프롬프트 & LLM
#    - 검색 결과를 context로 주입해 정확한 답을 유도
# ===========================================
rag_prompt = ChatPromptTemplate.from_messages([
    ("system",
     "너는 재료/용접 전문가야. 아래 문맥(context)을 근거로 질문에 답하되, "
     "모르면 모른다고 말해. 추측하지 마라.\n\n[CONTEXT]\n{context}\n"),
    ("user", "{question}")
])
llm = ChatOpenAI(model="gpt-3.5-turbo")
parser = StrOutputParser()

# ===========================================
# 7) RAG 질의 함수
# ===========================================
def rag_answer(question: str) -> str:
    # (1) 검색
    docs = retriever.get_relevant_documents(question)
    context = "\n\n".join([d.page_content for d in docs])
    # (2) 생성
    chain = rag_prompt | llm | parser
    return chain.invoke({"context": context, "question": question})

# ===========================================
# 8) 실행 예시
# ===========================================
q = "야외 시공에서 FCAW의 장점은 뭐야? 틈부식 관련 주의사항도 알려줘."
print(rag_answer(q))

```
</details>

In [10]:
### 여기에 정답을 작성해보세요
from dotenv import load_dotenv
import os
from langchain_text_splitters import CharacterTextSplitter        # 문서 청크 분할
from langchain_community.vectorstores import FAISS               # 벡터DB로 FAISS 사용
from langchain_openai import OpenAIEmbeddings                    # OpenAI 임베딩
from langchain_core.prompts import ChatPromptTemplate            # RAG 프롬프트
from langchain_openai import ChatOpenAI                          # LLM
from langchain_core.output_parsers import StrOutputParser        # 출력 파서

##  9. 최종 프로젝트

직접 챗봇을 기획하고 구현하는 프로젝트를 수행합니다.  
아래의 예시 중 하나를 선택하거나, 자신만의 주제를 정해도 좋습니다.

- 전공 Q&A 챗봇: 전공 분야의 질문에 자동으로 답변하는 챗봇
- 특정 문서 기반 검색 챗봇: 특정 보고서나 매뉴얼을 기반으로 정보를 찾아주는 챗봇
- FAQ 자동응답 챗봇: 자주 묻는 질문에 자동으로 응답하는 고객지원형 챗봇

**프로젝트 결과물**:  
  - 완성된 챗봇 웹 애플리케이션
    - 실제로 동작 가능한 형태의 챗봇 웹 서비스 구현  
    - **Streamlit**, **Flask**, **React** 등 어떤 프레임워크를 사용해도 무방

  - GitHub 저장소 혹은 실행파일
    - 구현한 코드와 실행 방법(`README.md`)을 포함하여 GitHub에 업로드 후 링크를 제출
    - 혹은 작성한 코드를 압축파일 형태로 제출
    - **Tip:** README.md에 챗봇 소개, 사용 방법, 기술 스택, 실행 예시 스크린샷을 포함하면 평가에 도움이 됩니다

**제출 방법**

- **제출 기한:** 수업 마지막날 밤 12시까지  
- **제출 이메일:** `rlaalstn1504@naver.com` 